In [3]:
import warnings
import pygame
from gtts import gTTS
import os
import torch
import datetime
import requests
import speech_recognition as sr
import tempfile

# Suppress FutureWarning for tokenization spaces
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")

In [14]:
# Load the model and tokenizer globally
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

In [23]:
# Select the microphone
def select_microphone(index):
    try:
        mic = sr.Microphone(device_index=index)
        return mic
    except Exception as e:
        print(f"Error selecting microphone: {e}")
        return None


In [24]:
# Initialize the pygame mixer for TTS
pygame.mixer.init()

# OpenWeatherMap API key
WEATHER_API_KEY = '6375f15162952d140f9908db1d7ffb99'  # Your API key


In [25]:
# Capture speech quickly
def capture_speech(mic):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300  # Faster detection

    with mic as source:
        recognizer.adjust_for_ambient_noise(source, duration=0.3)
        try:
            audio = recognizer.listen(source, timeout=2, phrase_time_limit=3)
            return recognizer.recognize_google(audio).lower()
        except (sr.WaitTimeoutError, sr.UnknownValueError, sr.RequestError):
            return None


In [26]:
# Listen for wake word
def listen_for_wake_word(mic):
    recognizer = sr.Recognizer()

    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.2)  # Increased duration for better tuning
            print("Waiting for wake word...")
            audio = recognizer.listen(source)

        try:
            user_input = recognizer.recognize_google(audio).lower()
            if "hey amigo" in user_input:
                print("Wake word detected!")  # Only print once
                return True  # Break the loop on wake word detection
        except sr.UnknownValueError:
            continue
        except sr.RequestError:
            continue


In [27]:
# Handle time, date, location, and weather queries
def handle_time_date_queries(user_input):
    if "time" in user_input:
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif "date" in user_input or "today" in user_input:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return f"Today is {current_date}."
    elif "location" in user_input:
        return get_current_location()
    elif "weather" in user_input:
        return get_current_weather()
    else:
        return None

# Get current location
def get_current_location():
    try:
        response = requests.get("http://ip-api.com/json")
        data = response.json()
        if data['status'] == 'success':
            city = data.get('city', 'unknown city')
            region = data.get('regionName', 'unknown region')
            country = data.get('country', 'unknown country')
            return f"You are in {city}, {region}, {country}."
        else:
            return "I couldn't determine your location."
    except Exception:
        return "Error fetching location."

# Get current weather
def get_current_weather():
    location = get_current_location()
    if "in" in location:
        city = location.split("in ")[1].split(",")[0]
        try:
            weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
            response = requests.get(weather_url)
            data = response.json()
            if data['cod'] == 200:
                weather_description = data['weather'][0]['description']
                temperature = data['main']['temp']
                return f"The current weather in {city} is {weather_description} with a temperature of {temperature}°C."
            else:
                return "Sorry, I couldn't get the weather information."
        except Exception:
            return "Error fetching weather."
    else:
        return "I couldn't determine the location for weather."


In [28]:
# Generate response
def generate_response(user_input):
    if "what is your name" in user_input:
        return "I am Amigo, your all-weather conversational companion."

    time_date_response = handle_time_date_queries(user_input)
    if time_date_response:
        return time_date_response

    if any(keyword in user_input for keyword in ["who is", "when is", "where is", "current", "today", "news"]):
        return "Thank you for your query, but that's out of my scope of training."

    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, max_length=50)
    reply_ids = model.generate(inputs['input_ids'], max_length=50, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(reply_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)


In [29]:
# Speak response using gTTS and pygame
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        temp_file_path = temp_audio_file.name
        tts.save(temp_file_path)

    # Play the audio using pygame
    pygame.mixer.music.load(temp_file_path)
    pygame.mixer.music.play()

    # Wait until the audio is done playing
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)  # Adding a delay to avoid busy-waiting

    # Clean up: stop music and remove the file only after playback is done
    pygame.mixer.music.stop()  # Ensure music is stopped before quitting
    if os.path.exists(temp_file_path):
        os.remove(temp_file_path)


In [30]:
# Main function
def amigo_conversational_companion():
    mic_index = 2  # Automatically set to microphone index 2

    # Select the microphone
    mic = select_microphone(mic_index)
    if mic is None:
        print("Failed to select microphone. Exiting...")
        return

    # Waiting for the wake word
    if listen_for_wake_word(mic):
        print("Wake word detected! Starting conversation.")
        greeting = "Hi, Hello, Namaskara, how can I assist you today?"
        print(f"Amigo: {greeting}")

        # Speak greeting
        speak_response(greeting)

        # Listening for user queries
        while True:
            user_input = capture_speech(mic)
            
            if user_input:
                print(f"You: {user_input}")  # Display user query
                bot_response = generate_response(user_input)
                print(f"Amigo: {bot_response}")  # Display Amigo response
                speak_response(bot_response)  # Speak Amigo response

            if "exit" in user_input or "goodbye" in user_input:
                farewell = "Goodbye, have a nice day!"
                print(f"Amigo: {farewell}")
                speak_response(farewell)
                break


In [31]:
if __name__ == "__main__":
    amigo_conversational_companion()

Waiting for wake word...
Wake word detected!
Wake word detected! Starting conversation.
Amigo: Hi, Hello, Namaskara, how can I assist you today?


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\prathap\\AppData\\Local\\Temp\\tmp1ujv8ql1.mp3'